In [1]:
# explicitly require this experimental feature
# from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
# from sklearn.ensemble import HistGradientBoostingClassifier
import pandas as pd
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,roc_auc_score,ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
from tensorflow import keras
from time import asctime


In [2]:
scaler=StandardScaler()

In [3]:
# logistic=MLPClassifier(hidden_layer_sizes=(256,256,128),max_iter=5,tol=1e-15,solver="adam",verbose=True,warm_start=True,n_iter_no_change=1000,learning_rate="adaptive")
logistic=MLPClassifier(hidden_layer_sizes=(512,512,512,512,512),max_iter=10,tol=1e-15,solver="adam",verbose=True,warm_start=True,n_iter_no_change=1000,learning_rate="adaptive",early_stopping=True,validation_fraction=0.20)

print("reading file")

reading file


In [4]:
train=pd.read_csv("train_transaction.csv")
# train=pd.read_pickle("cc_fraud_train_set.pkl")


In [5]:
print("file read.")
# cols=[i for i in train.columns if i.startswith("M") or i.startswith("D")]
df=train.copy()
categoricals=df.select_dtypes("object").columns.tolist()

file read.


In [6]:
print("fixing categoricals")
for obj_col in categoricals:
    temp=pd.get_dummies(df[obj_col])
    temp.columns=[f"{obj_col}_{col}" for col in temp.columns.tolist()]
    df=pd.concat([df.drop(columns=[obj_col]),temp],axis=1)
df.drop(columns=["TransactionID","TransactionDT"],inplace=True)
Xy=pd.DataFrame(df.values,columns=df.columns).drop(columns=["isFraud"])


fixing categoricals


In [7]:
Xy.head()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,C1,...,M5_F,M5_T,M6_F,M6_T,M7_F,M7_T,M8_F,M8_T,M9_F,M9_T
0,68.5,13926.0,NaN,150.0,142.0,315.0,87.0,19.0,NaN,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,29.0,2755.0,404.0,150.0,102.0,325.0,87.0,NaN,NaN,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,59.0,4663.0,490.0,150.0,166.0,330.0,87.0,287.0,NaN,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,50.0,18132.0,567.0,150.0,117.0,476.0,87.0,NaN,NaN,2.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50.0,4497.0,514.0,150.0,102.0,420.0,87.0,NaN,NaN,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 
Xy["fraud"]=train.isFraud.copy()
print("creating validation set")
validate=Xy.sample(frac=0.20) # data completely unseen
Xy=Xy[Xy.index.isin(validate.index)==False]


creating validation set


In [9]:
ones=ones=1/Xy.fraud.sum()
zeros=1/len(np.where(Xy.fraud==0)[0])
class_weights={0:zeros,1:ones}
rf=RandomForestClassifier(class_weight=class_weights)
sample=Xy.sample(frac=1)#was 0.20
sample_X=scaler.fit_transform(sample.drop(columns="fraud").fillna(0).values)
sample_X_df=pd.DataFrame(sample_X,columns=Xy.columns[:-1])

In [10]:
print("reducing float64s to round 8")
sample_X=np.round(sample_X,8)        
sample_y=sample.fraud

reducing float64s to round 8


In [11]:
print("getting correlations")
# corrs=pd.DataFrame(sample_X,columns=Xy.columns[:-1]).corr()
corrs=pd.DataFrame(Xy.drop(columns="fraud"),columns=Xy.columns[:-1]).corr()
print(asctime())

getting correlations
Wed Feb 16 20:00:48 2022


In [12]:
print("getting importances")
corrs_stack=corrs.index.tolist()
importances=pd.DataFrame(rf.fit(sample_X_df[corrs_stack],sample_y).feature_importances_.reshape(1,-1),columns=corrs_stack).T.reset_index().values.tolist()
print(asctime())

getting importances
Wed Feb 16 20:04:59 2022


In [13]:
corrs_stack=[i[0] for i in sorted(importances,key=lambda x:x[1],reverse=True) if i[1]!=0]
print("getting highly correlated fields")

for i in corrs_stack:
    test=corrs.loc[i].reset_index()
    test=test[abs(test[i])>0.80]
    if len(test[test["index"]!=i])>0:
        for r in test[test["index"]!=i]["index"].unique().tolist():
            try:
                corrs_stack.remove(r)
            except:
                print(len(corrs_stack))


getting highly correlated fields
523
518
444
444
444
444
444
444
444
444
444
441
441
441
441
441
441
433
430
430
429
427
415
403
403
397
394
390
390
388
388
388
387
387
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
385
382
382
382
382
382
382
382
382
382
382
382
364
363
360
360
360
360
359
356
355
355
353
352
344
338
338
338
338
338
338
335
335
324
324
323
323
323
316
316
315
310
305
305
305
305
305
305
305
305
305
303
303
302
302
301
301
301
301
301
301
300
300
300
298
298
298
298
298
298
298
298
296
296
296
296
295
295
295
295
294
285


In [14]:
df=df[corrs_stack]
Xy=Xy[corrs_stack+["fraud"]]


In [15]:
auc_stack=[]
val_auc_stack=[]
# scaler.fit(Xy[re_importances.field.values])
scaler.fit(Xy[corrs_stack])


StandardScaler()

In [16]:
# X=pd.DataFrame(scaler.transform(Xy[re_importances.field.values].fillna(0)),columns=Xy.columns[:-1])
X=pd.DataFrame(scaler.transform(Xy[corrs_stack].fillna(0)),columns=Xy.columns[:-1])
y=Xy.fraud.reset_index(drop=True)
# X_test=scaler.transform(validate[re_importances.field.values].fillna(0))
X_test=pd.DataFrame(scaler.transform(validate[corrs_stack].fillna(0)),columns=corrs_stack)
y_test=validate.fraud
X=np.round(X,8)

In [17]:
logistic=MLPClassifier(hidden_layer_sizes=(256,256,256),max_iter=10,tol=1e-15,solver="adam",learning_rate_init=0.001,verbose=True,warm_start=True,n_iter_no_change=1000,learning_rate="adaptive")#,early_stopping=True,validation_fraction=0.20

In [18]:
choices=[21,51,61,81,101]
p=[0.20,0.30,0.20,0.25,0.05]
auc_val_stack=[]
# !!!! set params accordingly
print("starting training")
choice=np.random.choice(choices,p=p)
sample_pct=1.0#np.random.randint(10,choice)/100
y_train=y.sample(frac=sample_pct)
X_train=X.loc[y_train.index]

starting training


In [19]:
model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(X_train.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               68864     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 200,705
Trainable params: 200,705
Non-trai

In [20]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    # keras.metrics.Precision(name="precision"),
    keras.metrics.AUC(name="auc"),
]


In [21]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)
callbacks = [keras.callbacks.EarlyStopping(monitor='val_auc', patience=200),keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}_val_scores_tp_{val_tp}_fn_{val_fn}_tn_{val_tn}_fp_{val_fp}_auc_{val_auc}.h5")]
class_weight = class_weights

In [22]:
model.fit(
    X_train.values,
    y_train.values,
    batch_size=2048,
    epochs=200,
    verbose=0,
    callbacks=callbacks,
    validation_data=(X_test.values, y_test.values),
    class_weight=class_weight,
)
